# notebooks.documentation
```
generative_models_via_sqsp
    |
    |_ utilities
        |
        |_ compiler.py
        |
        |_ grover_state_preparation.py
        |
        |_ kernels.py
        |
        |_ qcbm.py  
        |
        |_ quantum_gates.py    
        |
        |_ sampling.py
    |
    .
    .
    .  
  ```

In [1]:
import os
os.chdir('generative_models_via_sqsp/utilities')

In [2]:
import numpy as np
import scipy.sparse as sps

## compiler

### `compiler(ops, locs, n)`

Compiles operators into a specific Hilbert space.

**Args**
- `ops` (list or tuple): A list or tuple of operators to be applied to the system.
- `locs` (list or tuple): The qubit locations on which the operators will act.
- `n` (int): The total number of qubits in the quantum system.

**Returns**
- `scipy.sparse.csr_matrix`: The resulting sparse matrix after applying all operators.

In [3]:
from quantum_gates import sz, sx
from compiler import compiler

locs = [0, 1]
n = 2
result = compiler([sx, sz], locs, n)
print(result.todense())

[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]


### `_wrap_identity(data_list, num_bit_list)`

Helper function to apply identity operators to the Hilbert space.

**Args:**
- `data_list` (list): A list of operators to be applied to the quantum system.
- `num_bit_list` (list): A list containing the number of qubits on which each operator acts.

**Returns:**
- `scipy.sparse.csr_matrix`: The resulting sparse matrix after applying the operators.

**Raises:**
- `Exception`: If the length of `num_bit_list` is inconsistent with the number of operators.


### `initial_wf(num_bit)`


Generates the initial wave function |00...0> for a quantum system.

**Args:**
- `num_bit` (int): The number of qubits in the system.

**Returns:**
- `np.ndarray`: The initial wave function as a numpy array.

**Remarks:**
- The function returns the state vector |00...0>, which represents the quantum system being initialized to the ground state (all qubits in state |0>).

## grover_state_preparation

### `get_grover_angles(p_i_set, m)`


Calculates Grover's angles for a given set of probabilities.

**Args:**
- `p_i_set` (list or array-like): A list of probabilities corresponding to quantum states.
- `m` (int): The number of qubits used in the quantum circuit.

**Returns:**
- `list`: A list of rotation angles required for Grover's state preparation.

**Raises:**
- `ValueError`: If the computed number of angles does not match the required length for `m` qubits.

### `state_expansion(m, thetas)`

Constructs a quantum circuit that applies rotations based on the calculated angles.

**Args:**
- `m` (int): The number of qubits in the circuit.
- `thetas` (list): A list of rotation angles computed for Grover's algorithm.

**Returns:**
- `QuantumCircuit`: A quantum circuit implementing the state preparation.

**Raises:**
- `ValueError`: If the number of angles does not match the required `2^m - 1`.

## kernels

### `mix_rbf_kernel(x, y, sigma_list)`

Computes a mixture of RBF kernels between two datasets.

**Args:**
- `x` (numpy.ndarray): Dataset `x`, shape `(n_samples_x, n_features)`.
- `y` (numpy.ndarray): Dataset `y`, shape `(n_samples_y, n_features)`.
- `sigma_list` (list or np.ndarray): List of sigma values for the RBF kernels.

**Returns:**
- `numpy.ndarray`: The kernel matrix computed between `x` and `y`.

**Raises:**
- `ValueError`: If any sigma values are non-positive.

### `RBFMMD2` class


Computes the squared Maximum Mean Discrepancy (MMD) using an RBF kernel.

**Methods**
- `__init__(sigma_list)`
Initializes the RBFMMD2 object with a list of sigma values.
- `compute(x, y)`
Computes the squared MMD between two datasets.

In [4]:
from kernels import RBFMMD2

x = np.array([[1], [2], [3]])
y = np.array([[1], [2], [3]])
sigma_list = [1.0]
rbfmmd = RBFMMD2(sigma_list)
mmd_value = rbfmmd.compute(x, y)
print(f"Squared MMD value: {mmd_value}")

Squared MMD value: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## qcbm

### `ArbitraryRotation` class

Represents a quantum gate that applies arbitrary rotations to qubits in the quantum circuit. It can apply three rotations per qubit, represented by a list of rotation angles.

**Methods**
- `__init__(self, num_bit)`: Initializes the ArbitraryRotation instance with the specified number of qubits.
- `num_param`: Property that returns the number of parameters for the rotations (3 parameters per qubit).
- `toscr(self, theta_list)`: Transforms this block into a sequence of sparse CSR matrices based on the provided list of rotation angles.

In [5]:
from qcbm import ArbitraryRotation

num_bit = 2
rotation_gate = ArbitraryRotation(num_bit)
print(f"Number of parameters in ArbitraryRotation: {rotation_gate.num_param}")
theta_list = np.random.rand(6)
csr_matrices = rotation_gate.tocsr(theta_list)
for i, mat in enumerate(csr_matrices):
    print(f"CSR matrix {i} for rotation:")
    print(mat.toarray())

Number of parameters in ArbitraryRotation: 6
CSR matrix 0 for rotation:
[[ 0.93506409-0.33137603j -0.00388852-0.12581717j  0.        +0.j
   0.        +0.j        ]
 [ 0.00388852-0.12581717j  0.93506409+0.33137603j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.93506409-0.33137603j
  -0.00388852-0.12581717j]
 [ 0.        +0.j          0.        +0.j          0.00388852-0.12581717j
   0.93506409+0.33137603j]]
CSR matrix 1 for rotation:
[[ 0.86845199-0.48563817j  0.        +0.j          0.02881078-0.0954812j
   0.        +0.j        ]
 [ 0.        +0.j          0.86845199-0.48563817j  0.        +0.j
   0.02881078-0.0954812j ]
 [-0.02881078-0.0954812j   0.        +0.j          0.86845199+0.48563817j
   0.        +0.j        ]
 [ 0.        +0.j         -0.02881078-0.0954812j   0.        +0.j
   0.86845199+0.48563817j]]


### `CNOTEntangler` class

Applies a series of CNOT gates that entangle pairs of qubits. The entanglement is formed by applying a CNOT gate for each pair in the provided list.

**Methods**
- `__init__(self, num_bit, pairs)`: Initializes the CNOTEntangler instance with the number of qubits and the pairs for entangling.
  
- `num_param`: Property that returns the number of parameters (CNOT gates do not have parameters).

- `toscr(self, theta_list)`: Transforms this block into a sequence of sparse CSR matrices by applying CNOT gates to the specified qubit pairs.

In [6]:
from qcbm import CNOTEntangler

num_bit = 2
pairs = [(0, 1)]
cnot_entangler = CNOTEntangler(num_bit, pairs)
print(f"Number of parameters in CNOTEntangler: {cnot_entangler.num_param}")
theta_list = np.array([])
csr_matrices = cnot_entangler.tocsr(theta_list)
print("CNOTEntangler CSR matrix:")
print(csr_matrices[0].toarray())


Number of parameters in CNOTEntangler: 0
CNOTEntangler CSR matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


### `BlockQueue` class


Keeps track of the quantum circuit's evolution by managing blocks of operations and applying them to a quantum register while efficiently tracking the parameter changes.

**Methods**

- `__init__(self, *args)`: Initializes a BlockQueue instance with a sequence of quantum operations (blocks).
  
- `__call__(self, qureg, theta_list)`: Applies operations to the quantum register in place using the provided list of parameters.

- `num_bit`: Property that returns the number of qubits in the quantum circuit.

- `num_param`: Property that returns the total number of parameters across all blocks.

In [7]:
from qcbm import BlockQueue

num_bit = 2
pairs = [(0, 1)]
blocks = [ArbitraryRotation(num_bit), CNOTEntangler(num_bit, pairs)]
block_queue = BlockQueue(blocks)
print(f"Number of qubits: {block_queue.num_bit}")
print(f"Total number of parameters in BlockQueue: {block_queue.num_param}")
qureg = np.zeros(2**num_bit, dtype='complex128')
qureg[0] = 1.0
theta_list = np.random.rand(block_queue.num_param)
block_queue(qureg, theta_list)
print("Updated quantum register (wavefunction):")
print(qureg)

Number of qubits: 2
Total number of parameters in BlockQueue: 6
Updated quantum register (wavefunction):
[ 0.2659137 -0.86411532j -0.03074796-0.00140767j -0.21693033-0.36086013j
 -0.04182868-0.05117384j]


### `QCBM` class


The Quantum Circuit Born Machine framework that learns to approximate probability distributions using quantum circuits. The model uses rotation gates and CNOT entanglers, and the optimization is performed using the MMD loss function.

**Methods**
- `__init__(self, circuit, mmd, p_data, batch_size=None)`: Initializes the QCBM instance with the specified quantum circuit, MMD metric, target probability distribution, and batch size (optional).

- `depth`: Property that returns the depth of the circuit, defined by the number of entanglers.

- `pdf(self, theta_list)`: Gets the probability distribution function by applying the quantum circuit to the quantum state.

- `mmd_loss(self, theta_list)`: Computes the MMD loss for the given parameters.

- `gradient(self, theta_list)`: Computes the gradient of the MMD loss with respect to the parameters using numerical gradient computation.

In [8]:
from qcbm import ArbitraryRotation, CNOTEntangler, BlockQueue, QCBM

def dummy_mmd(prob1, prob2):
    return np.mean((prob1 - prob2) ** 2)

blocks = [ArbitraryRotation(num_bit), CNOTEntangler(num_bit, pairs)]
block_queue = BlockQueue(blocks)
p_data = np.random.rand(2**num_bit)
p_data /= p_data.sum()
qcbm = QCBM(circuit=block_queue, mmd=dummy_mmd, p_data=p_data)
theta_list = np.random.rand(block_queue.num_param)
prob_distribution = qcbm.pdf(theta_list)
print("Computed probability distribution:")
print(prob_distribution)

Computed probability distribution:
[0.84623881 0.0032512  0.12921815 0.02129184]


## quantum_gates


In [9]:
from quantum_gates import _ri, sz

theta = np.pi / 4
rotation_sz = _ri(sz, theta)
print(rotation_sz.todense())

[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]


In [10]:
from quantum_gates import rot

t1, t2, t3 = np.pi/4, np.pi/4, np.pi/4
full_rotation = rot(t1, t2, t3)
print(full_rotation.todense())

[[0.65328148-0.65328148j 0.        -0.38268343j]
 [0.        -0.38268343j 0.65328148+0.65328148j]]


In [11]:
from quantum_gates import CNOT

ibit = 0
jbit = 1
n = 2
cnot_result = CNOT(ibit, jbit, n)
print(cnot_result.todense())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


## sampling

### `sample_from_prob` function

Samples `num_sample` elements from the dataset `x` based on the given probability distribution `pl`. The probabilities are normalized to ensure they sum to 1 before sampling. It returns the sampled elements from `x`.

**Args**
- `x` (numpy.ndarray): Dataset `x` from which to sample, shape `(n_samples, n_features)`.
- `pl` (numpy.ndarray): Probability distribution over the dataset `x`, shape `(n_samples,)`.
- `num_sample` (int): The number of samples to draw.

**Returns**
- `numpy.ndarray`: The sampled elements from `x`, shape `(num_sample, n_features)`.

In [12]:
from sampling import sample_from_prob

x = np.array([[1], [2], [3], [4], [5]])
pl = np.array([0.1, 0.2, 0.3, 0.2, 0.2])
num_sample = 3
samples = sample_from_prob(x, pl, num_sample)
print(f"Sampled data points (Test Case 1): {samples}")

Sampled data points (Test Case 1): [[3]
 [2]
 [4]]


### `prob_from_sample` function

Computes the empirical probability distribution from a dataset. It counts the occurrences of each element in the dataset and normalizes the counts to produce a probability distribution.

**Args**
- `dataset` (numpy.ndarray): The dataset to compute the probability distribution from.
- `hndim` (int): The number of possible distinct outcomes in the dataset.

**Returns**
- `numpy.ndarray`: The empirical probability distribution, shape `(hndim,)`.

In [13]:
from sampling import prob_from_sample

dataset = np.array([0, 1, 2, 1, 3, 0, 0, 1, 3, 2])
hndim = 4
empirical_prob = prob_from_sample(dataset, hndim)
print(f"Empirical probability distribution (Test Case 3): {empirical_prob}")

Empirical probability distribution (Test Case 3): [0.3 0.3 0.2 0.2]
